# Categorização de avaliações

**Passo a passo:**

1. **Pré-processamento**: Remover stop words e aplicar stemming ou lematização
2. **Extração de Características**: Usar TF-IDF ou Word2Vec para criar representações vetoriais das avaliações
3. **Clusterização**: Aplicar K-means para definir categorias com base nas avaliações
4. **Classificação**: Usar o modelo para categorizar novas avaliações de acordo com as categorias definidas
5. **Avaliação e Ajuste**: Medir a precisão e otimizar o pipeline

## 1. Imports

In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import silhouette_score

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import gensim
from gensim.models import Word2Vec

### 1.1. Baixar stopwords e pacotes necessários do NLTK (Natural Language Toolkit)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt') # punkt sequence tokenizer = divide um texto em uma lista de sentenças usando um algoritmo não supervisionado para construir um modelo  para abreviação de palavras
nltk.download('wordnet')

## 2. Pre-processamento do texto

### 2.1. Remoção de stop words
Preposições, conjunções, artigos, etc

In [ ]:
stop_words = set(stopwords.words('portuguese'))

### 2.2. Stemming
Abordagem de simplificação de palavras:
- Stemming: reduz palavras ao seu radical, ignorando as regras linguísticas (é útil para reduzir a dimensionalidade)
- Lematização: Reduz palavras à sua forma básica, considerando o contexto gramatical (é mais precisa)

In [ ]:
stemmer = SnowballStemmer("portuguese")

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and word.isalpha()]
    return " ".join(tokens)

df['preprocessed_text'] = df['review_column'].apply(preprocess_text)

## 3. Extração de características

### 3.1. Vetorização com TF-IDF

TF-IDF (term frequence - inverse document frequency) calcula a relevância de cada termo dentro de um documento

O TfidfVectorizer representa as availações:

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000)
X_tfidf = vectorizer.fit_transform(df['preprocessed_text'])

### 3.2 Embeddings com Word2Vec

O Word2Vec captura relações semânticas entre as palavras, criando representações vetoriais densas, que são úteis para capturar o contexto semântico (no entanto, demanda mais recursos computacionais)

In [ ]:
tokenized_reviews = df['preprocessed_text'].apply(lambda x: x.split())
word2vec_model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=2, sg=1)

Para obter a representação de um documento, podemos tirar a média dos vetores Word2Vec das palavras:

In [ ]:
def get_average_word2vec(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

df['word2vec_vector'] = df['preprocessed_text'].apply(lambda x: get_average_word2vec(x.split(), word2vec_model))

## 4. Clusterização para a criação de categorias

### 4.1 K-Means clustering
Usando TF-IDF ou Word2Vec, é possível aplicar o algoritmo de K-means para agrupar as avaliações em categorias

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X_tfidf)  # ou X_word2vec se usar Word2Vec

O Silhouette Score mostra o número ideal de clusters

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X_tfidf, kmeans.labels_)

### 4.2. Definição dos clusters
Após definir os clusters, as categorias podem ser interpretadas extraindo as palavras mais representativas de cada cluster
No TF-IDF, isso é feito analisando os pesos de cada termo em cada clsuter

## 5. Classificação das avaliações nas categorias
O modelo K-means prevê o cluster mais próximo para categorizar as novas avaliações em uma ou mais categorias

Se uma avaliação puder pertencer a várias categorias, considere usar uma abordagem de similaridade de cosseno para encontrar as categorias mais próximas.

### 5.1. Vetorização e classificação da avaliação

Para cada nova avaliação, é necessário:
1. Pré-processar o texto com a mesma função criada
2. Transformar o texto em vetor usando Tf-IDF ou Word2Vec
3. Calcular a similaridade com os clusters e categorizar a avaliação de acordo com as categorias mais próximas

Exemplo de nova avaliação:

In [ ]:
new_review = "Avaliação de exemplo"
preprocessed_review = preprocess_text(new_review)
review_vector = vectorizer.transform([preprocessed_review])  # usando TF-IDF

# Prever o cluster
predicted_cluster = kmeans.predict(review_vector)

## 6. Avaliação e ajuste do modelo

Após implementar a classificação, a precisão pode der avaliada com `accuracy` ou `f1-score`, dependendo de como a tarefa está estruturada. Para melhorar a precisão, podemos ajustar o número de clusters, alterar o pré-processamento ou experimentar outras técnicas de vetorização para melhorar a precisão